In [26]:
import pandas as pd
import psycopg2
import sqlalchemy
from psycopg2 import Error
from sqlalchemy import create_engine
from dotenv import load_dotenv, dotenv_values
from fastapi import Depends, HTTPException
from sqlalchemy import create_engine, text
from sqlalchemy.orm import Session
import hashlib
import os
from decimal import Decimal
load_dotenv()

True

In [2]:
# PostgreSQL connection details
username = os.getenv('DB_USERNAME')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')
database = os.getenv('DB_DATABASE')

In [3]:
connection = psycopg2.connect(
    user=username,
    password=password,
    host=host,
    port=port,
    database=database,
sslmode = 'require'
)

In [4]:
# cursor = connection.cursor()
# table_query = """
# CREATE TABLE users (
#     id Serial PRIMARY KEY ,
#     username TEXT NOT NULL UNIQUE,
#     apikey TEXT NOT NULL,
#     credit_points INTEGER DEFAULT 500  
# );
# """

# cursor.execute(table_query)

In [14]:
# Modify the data type of credit_points to NUMERIC(10, 7) in PostgreSQL
with Session(engine) as session:
    session.execute(text("ALTER TABLE users ALTER COLUMN credit_points TYPE NUMERIC(15, 7)"))
    session.commit()

In [10]:
engine.connect().commit()

In [15]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT column_name, data_type, numeric_precision, numeric_scale "
                               "FROM information_schema.columns "
                               "WHERE table_name = 'users' AND column_name = 'credit_points'")).fetchall()

    for row in result:
        print(row)


('credit_points', 'numeric', 15, 7)


In [6]:
# connection.rollback()

In [8]:
# connection.commit()

In [4]:
def hash_api_key(api_key : str):
    return hashlib.sha256(api_key.encode()).hexdigest()

In [8]:
def insert_data(cursor, username: str, api_key: str, credit: int = 5000):
    # Assuming hash_api_key is a function that hashes the API key
    hashed_api_key = hash_api_key(api_key)
    
    q = """
    INSERT INTO users (username, apikey, credit_points) 
    VALUES (%s, %s, %s)
    """
    try:
        # Execute the query
        cursor.execute(q, (username, hashed_api_key, credit))
        
        # Optionally, commit the transaction if needed
        cursor.connection.commit()
    except:
        cursor.connection.rollback()
        return f"user {username} failed to insert"

    return f"User {username} inserted successfully with {credit} credits."


In [19]:
users =[
    ('otto', '45a7f6c8d9012e3f4b5a6c7d8e9f1a2b'),
    ('user1', '1a2b3c4d5e6f7g8h9i0j1k2m3n4o5p'),
    ('io','7c6a5e4d3b2a1f9e8d7c6r5t4y3u2i'),
    ('user3','9f8e7d6c5b4a3g2h1i0j9k8l7m6n5o')
]


"""
45a7f6c8d9012e3f4b5a6c7d8e9f1a2b
1a2b3c4d5e6f7g8h9i0j1k2m3n4o5p
7c6a5e4d3b2a1f9e8d7c6r5t4y3u2i
9f8e7d6c5b4a3g2h1i0j9k8l7m6n5o

""" 
''

''

In [24]:
# for user in users[1:]:
#     insert_data(cursor, user[0], user[1])

In [12]:
connection.rollback()

In [10]:
cursor = connection.cursor()

In [11]:
q2 = 'SELECT * From users'
cursor.execute(q2)
connection.commit()


In [12]:
row = cursor.fetchall()
row

[(1,
  'otto',
  '3634c59eb22d2003a453ad644a694052b8eab9f351bd1ad994aeb56e5e937f9a',
  5000),
 (2,
  'user1',
  'ac26e678c1d1ccc291bbffded36b70aa9927826ec4265bf4e5a7655913b9ada6',
  5000),
 (3,
  'io',
  'ed3e808c45aa85a5f036518442ea1876302402f8b69935c2e59f90c280d716b4',
  5000),
 (4,
  'user3',
  'fdba662617e2d6846a2796847fe8094f656fa78f45ba194af0203bc8c9138fc7',
  5000)]

In [5]:
# Setup database engine (SQLite example)
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')

In [20]:


# Setup database engine (SQLite example)
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')

# User authentication function
def authenticate_user(api_key: str):
    hashed_key = hash_api_key(api_key)
    query = "SELECT * FROM users WHERE apikey = :apikey"
    # if username:
    #     query += " AND username = :username"

    with engine.connect() as conn:
        user = conn.execute(text(query), {"apikey": hashed_key}).fetchone()

    if not user:
        return False
        raise HTTPException(status_code=401, detail="Invalid API key or user")
    
    return user[0], user[1], user[3]  # Returns user row


In [22]:
authenticate_user(users[0][1])

(1, 'otto', Decimal('5000.0000000'))

In [ ]:
from sqlalchemy.orm import Session

def update_user_credits_orm(engine, user_id: int, new_credits: int):
    with Session(engine) as session:
        # Fetch the user record
        user = session.execute(
            text("SELECT * FROM users WHERE id = :id"), {"id": user_id}
        ).fetchone()

        # Check if the user exists, then update
        if user:
            session.execute(
                text("UPDATE users SET credit_points = :new_credits WHERE id = :user_id"),
                {"new_credits": new_credits, "user_id": user_id}
            )
            session.commit()  # Commit to save the changes

# Call the function
update_user_credits_orm(engine, user_id=2, new_credits=4000)


In [25]:
# def deduct_credits(engine, user_id: int, row_count: int):
#     points_to_deduct = row_count  # For example, 1 credit point per row returned
    
#     with engine.connect() as conn:
#         user = conn.execute(text("SELECT credit_points FROM users WHERE id = :id"), {"id": user_id}).fetchone()

#         if user and user[0] >= points_to_deduct:
#             new_credits = user[0] - points_to_deduct
#             print(new_credits)
#             conn.execute(text("UPDATE users SET credit_points = :new_credits WHERE id = :id"),
#                          {"new_credits": new_credits, "id": user_id})
#             print ('deducted')
#         else:
#             raise HTTPException(status_code=402, detail="Insufficient credit points")


In [27]:


def deduct_credits(engine, user_id: int, row_count: int):
    points_to_deduct = Decimal(row_count /10000) # For example, 1 credit point per row returned
    with Session(engine) as session:
        # Fetch the user record and their current credit points
        user = session.execute(
            text("SELECT credit_points FROM users WHERE id = :id"), {"id": user_id}
        ).fetchone()

        # Check if the user exists and has enough credit points
        if user and user[0] >= points_to_deduct:
            new_credits = user[0] - points_to_deduct
            print(f"New Credits after deduction: {new_credits}")
            
            # Update user's credit points
            session.execute(
                text("UPDATE users SET credit_points = :new_credits WHERE id = :user_id"),
                {"new_credits": new_credits, "user_id": user_id}
            )
            
            # Commit the transaction to save the changes
            session.commit()
            print('Credits deducted successfully.')
        else:
            # If the user does not exist or has insufficient credits
            raise HTTPException(status_code=402, detail="Insufficient credit points")



In [24]:
def check_user_credits(engine, user_id: int):
    with engine.connect() as conn:
        result = conn.execute(text("SELECT credit_points FROM users WHERE id = :user_id"), {"user_id": user_id}).fetchone()
        if result:
            print(f"User ID {user_id} has {result[0]} credit points.")
        else:
            print(f"User ID {user_id} not found.")

# Call the function to verify the update
check_user_credits(engine, user_id=2)

User ID 2 has 3995.0000000 credit points.


In [28]:

# Example usage:
deduct_credits(engine, user_id=2, row_count=5)

New Credits after deduction: 3994.999499999999999999989592
Credits deducted successfully.


In [18]:
 user = engine.connect().execute(text("SELECT credit_points, username FROM users WHERE id = :id"), {"id": 2}).fetchone()
user

(5000, 'user1')

In [29]:
res = engine.connect().execute(text("select * from users")).fetchall()
res

[(1, 'otto', '3634c59eb22d2003a453ad644a694052b8eab9f351bd1ad994aeb56e5e937f9a', Decimal('5000.0000000')),
 (3, 'io', 'ed3e808c45aa85a5f036518442ea1876302402f8b69935c2e59f90c280d716b4', Decimal('5000.0000000')),
 (4, 'user3', 'fdba662617e2d6846a2796847fe8094f656fa78f45ba194af0203bc8c9138fc7', Decimal('5000.0000000')),
 (2, 'user1', 'ac26e678c1d1ccc291bbffded36b70aa9927826ec4265bf4e5a7655913b9ada6', Decimal('3994.9995000'))]

In [ ]:
res.fetchall()

User ID 2 has 4000 credit points.
